In [68]:
from youtube_tools import MyYouTubeAPI
from yt_dlp import YoutubeDL
import requests
import re
from bs4 import BeautifulSoup as bs

def get_video_id(url):
    return url.split('=')[-1].split('&')[0]

def get_channel_id(video_id):
    url = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {}
    with YoutubeDL(ydl_opts) as ydl:
        try:
            meta = ydl.extract_info(url, download=False)
        except:
            print("\nerror")
            exit(1)
    return meta['channel_id']

def get_playlist_id_of_uploads(myt, channel_id):
    return 'UU' + channel_id[2:]

def get_iframe(video_id):
    return f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'

def get_playlist_info(video_id, playlist_id):
    #ua = 'ozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'
    #headers = {'User-Agent': ua}
    url = f'https://www.youtube.com/watch?v={video_id}&list={playlist_id}'
    res = requests.get(url)
    soup = bs(res.content,'html.parser')
    index = int(re.search(r'currentIndex":(\d)',str(soup)).group(1))
    video_count = int(re.search(r'totalVideosText":{"runs":\[{"text":"(\d*)', str(soup)).group(1))
    print(index, video_count)
    return index, video_count

# あとは指定した動画のインデックスさえ特定できれば...
def get_sides(playlist_id, index):
    url = f'https://www.youtube.com/playlist?list={playlist_id}'
    # 範囲は0及びアイテム数を超えた場合無視
    ydl_opts = {'playlist_items': f'{index-1}-{index+1}'}
    with YoutubeDL(ydl_opts) as ydl:
        try:
            meta = ydl.extract_info(url, download=False)
        except:
            print("\nerror")
            exit(1)
    video_ids_sides = [items['id'] for items in meta['entries']]
    return video_ids_sides

def get_dic(myt, url):
    video_id = get_video_id(url)
    channel_id = get_channel_id(myt, video_id)
    playlist_id = get_playlist_id_of_uploads(myt, channel_id)
    index, video_count = get_playlist_info(video_id, playlist_id)
    sides = get_sides(playlist_id, index)
    dic = {}

    if index != 0: dic['next'] = sides[0]
    dic['selected'] = sides[1]
    if index != video_count: dic['pre'] = sides[2]

    return dic

In [69]:
print(get_channel_id('oj2DBTW43lI'))

[youtube] oj2DBTW43lI: Downloading webpage
[youtube] oj2DBTW43lI: Downloading android player API JSON
UCG0rzBZV_QMP4MtWg6IjhEA
